In [ ]:
# 자신이 코랩 gpu 사용중인지 체크하고 진행하자
!nvidia-smi

Fri Dec 22 00:29:40 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 이미지 파일들의 이름을 저장
from glob import glob # 폴더 파일명을 관리하는 패키지
import json # json 파일에서 정보 추출용 패키지
from PIL import Image
import os
import json
import shutil
import random
random.seed(1) # 랜덤의 시드 고정

In [ ]:
# 이미지들을  416x416으로 resize 하기
from PIL import Image
def resize_and_delete_original(folder_path, target_size=(416, 416)):
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            # 이미지 파일 경로
            image_path = os.path.join(folder_path, filename)
            # 이미지 열기
            with Image.open(image_path) as img:
                # 이미지 resize
                resized_img = img.resize(target_size, Image.LANCZOS)
                # 리사이즈된 이미지 저장 (원본 이미지를 덮어쓰기)
                resized_img.save(image_path)

# yolo의 경우에는 32의 배수의 이미지 사이즈를 처리 할 수 있으니 이미지파일들의 리사이징을 통한 크기 통일이 필요하다.
# 라벨링 이미지들이 400*400으로 통일 되어 있으니 최소한의 사이즈 변경을 위해 416*416을 default로 지정한다.
def resize_and_copy_images(img_path, output_folder, target_size=(416, 416)):
    new_image_path = os.path.join(output_folder, os.path.basename(img_path))
    # 이미지 열기
    with Image.open(img_path) as img:
        # 이미지 resize
        resized_img = img.resize(target_size, Image.LANCZOS)
        # 리사이즈된 이미지 저장
        resized_img.save(new_image_path)

In [ ]:
def convert_to_yolo_format_dog_BigSmall(json_data):
    yolo_format_data = []

    # 질병마다 1개씩 모델 만들때는 class가 0-2, 0-3 은 안돼고 0-1 방식이여야 한다
    label_mapping = {
        "결막염": {"대분류": 0, "소분류": {"무": 0, "유": 1}},
        "궤양성각막질환": {"대분류": 1, "소분류": {"무": 0, "상": 1, "하": 1}},
        "백내장": {"대분류": 2, "소분류": {"무": 0, "비성숙": 1, "성숙": 1, "초기": 1}},
        "비궤양성각막질환": {"대분류": 3, "소분류": {"무": 0, "상": 1, "하": 1}},
        "색소침착성각막염": {"대분류": 4, "소분류": {"무": 0, "유": 1}},
        "안검내반증": {"대분류": 5, "소분류": {"무": 0, "유": 1}},
        "안검염": {"대분류": 6, "소분류": {"무": 0, "유": 1}},
        "안검종양": {"대분류": 7, "소분류": {"무": 0, "유": 1}},
        "유루증": {"대분류": 8, "소분류": {"무": 0, "유": 1}},
        "핵경화": {"대분류": 9, "소분류": {"무": 0, "유": 1}},
    }
    image_info = json_data["images"]["meta"]
    image_width, image_height = image_info["width_height"]

    label_info = json_data["label"]
    label_filename = label_info["label_filename"]
    label_category_id = label_info["label_category_id"]
    label_bbox = label_info["label_bbox"]

    class_info = label_mapping.get(label_info["label_disease_nm"])
    if class_info:
        label_bbox = [float(coord) for coord in label_bbox]
        image_width = float(image_width)
        image_height = float(image_height)
        x_center = ((label_bbox[0] + label_bbox[2]) / (2 * image_width))
        y_center = ((label_bbox[1] + label_bbox[3]) / (2 * image_height))
        width = ((label_bbox[2] - label_bbox[0]) / image_width)
        height = ((label_bbox[3] - label_bbox[1]) / image_height)
        # 라벨링 박스 데이터가 이미지 크기보다 크거나 작은 경우 대처
        if width < 0:
          width = 0
        if height < 0:
          height = 0
        if width > 1:
          width = 1
        if height > 1:
          height = 1
        # 0 ~ 1 사이의 값을 가져야 하지만 -0.00043~~ , 1.000076~~ 등 근소하게 범위를 넘어서므로 0, 1로 값을 대체한다.
        # 만약 대체하지 않으면 yolo 모델 학습 이전에 데이터를 로드 및 확인하는 과정에서
        # 부적절한 범위의 데이터들이 버려지며 학습 시작까지의 시간이 좀 더 소요된다.

        sub_class_label = class_info["소분류"].get(label_info["label_disease_lv_3"])
        yolo_format_data.append(f"{sub_class_label} {x_center} {y_center} {width} {height}")
    return yolo_format_data

def save_yolo_format_to_txt_dog_BigSmall(input_folder, output_folder):
    # JSON 파일 읽기
    with open(input_folder, "r", encoding='utf-8') as json_file:
        json_data = json.load(json_file)
    # YOLO 형식으로 변환
    yolo_data = convert_to_yolo_format_dog_BigSmall(json_data)
    # YOLO 형식의 텍스트 파일 생성
    txt_label_path = os.path.join(output_folder, os.path.splitext(os.path.basename(input_folder))[0] + ".txt")
    with open(txt_label_path, "w") as txt_label_file:
        txt_label_file.write("".join(yolo_data[0]))

def json_to_txt_And_copy_images_Dog(input_folder, output_folder_label, output_folder_images):
    for filename in os.listdir(input_folder):
      if filename.endswith(".json"):
        json_path = os.path.join(input_folder, filename)
        save_yolo_format_to_txt_dog_BigSmall(json_path, output_folder_label)
      if filename.endswith((".jpg",".png")):
        img_path = os.path.join(input_folder, filename)
        resize_and_copy_images(img_path, output_folder_images, target_size=(416, 416))


In [ ]:
def split_data(input_folder, test_folder, ratio=0.875):
    # 입력 폴더 내 모든 파일 목록을 가져옵니다.
    all_files = os.listdir(input_folder)
    random.shuffle(all_files)  # 목록을 무작위로 섞습니다.

    # 주어진 비율을 기반으로 훈련 및 테스트에 사용될 파일 수를 계산합니다.
    num_train = int(len(all_files) * ratio)
    num_test = len(all_files) - num_train

    # 파일을 훈련 및 테스트 세트로 나눕니다.
    test_files = all_files[num_train:]

    # test용 파일을 출력 폴더로 이동합니다.
    for file_name in test_files:
        source_path = os.path.join(input_folder, file_name)
        dest_path = os.path.join(test_folder, file_name)
        shutil.move(source_path, dest_path)


In [ ]:
# unzip 및 폴더명 변경 이후에 각 질병의 폴더명들이다.
# 모든 데이터를 한번에 학습시키기 위한 폴더로 전처리 및 이동시에는 바로 실행시키고 넘어가고, (시간은 오래 걸릴 것이다.)
# 원하는 1개의 질병의 데이터만 옮길 경우에는 주석으로 달아 놓은 숫자를 잘 보기 바란다.
dog_list_train_vm_2 = [ # 개 질병 10종 모든 폴더
        '/content/dog/eye/normal/0/mu', # 0 - 결막염 # 0
        '/content/dog/eye/normal/0/yu', # 1
        '/content/dog/eye/normal/1/ha', # 2
        '/content/dog/eye/normal/1/mu', # 1 - 궤양성각막질환 # 3
        '/content/dog/eye/normal/1/san', # 4
        '/content/dog/eye/normal/2/beSeungSuk', # 5
        '/content/dog/eye/normal/2/choGi', # 6
        '/content/dog/eye/normal/2/mu', # 2 - 백내장 # 7
        '/content/dog/eye/normal/2/seungSuk', # 8
        '/content/dog/eye/normal/3/ha', # 9
        '/content/dog/eye/normal/3/mu', # 3 - 비궤양성각막질환 # 10
        '/content/dog/eye/normal/3/san', # 11
        '/content/dog/eye/normal/4/mu', # 4 - 색소침착성각막염 # 12
        '/content/dog/eye/normal/4/yu', # 13
        '/content/dog/eye/normal/5/mu', # 5 - 안검내반증 # 14
        '/content/dog/eye/normal/5/yu', # 15
        '/content/dog/eye/normal/6/mu', # 6 - 안검염 # 16
        '/content/dog/eye/normal/6/yu', # 17
        '/content/dog/eye/normal/7/mu', # 7 - 안검종양 # 18
        '/content/dog/eye/normal/7/yu', # 19
        '/content/dog/eye/normal/8/mu', # 8 - 유루증 # 20
        '/content/dog/eye/normal/8/yu', # 21
        '/content/dog/eye/normal/9/mu', # 9 - 핵경화 # 22
        '/content/dog/eye/normal/9/yu', # 23
]

dog_list_valid_vm_2 = [ # 개 질병 10종 모든 폴더
        '/content/val/dog/eye/normal/0/mu', # 0 - 결막염
        '/content/val/dog/eye/normal/0/yu',
        '/content/val/dog/eye/normal/1/ha',
        '/content/val/dog/eye/normal/1/mu', # 1 - 궤양성각막질환
        '/content/val/dog/eye/normal/1/san',
        '/content/val/dog/eye/normal/2/beSeungSuk',
        '/content/val/dog/eye/normal/2/choGi',
        '/content/val/dog/eye/normal/2/mu', # 2 - 백내장
        '/content/val/dog/eye/normal/2/seungSuk',
        '/content/val/dog/eye/normal/3/ha',
        '/content/val/dog/eye/normal/3/mu', # 3 - 비궤양성각막질환
        '/content/val/dog/eye/normal/3/san',
        '/content/val/dog/eye/normal/4/mu', # 4 - 색소침착성각막염
        '/content/val/dog/eye/normal/4/yu',
        '/content/val/dog/eye/normal/5/mu', # 5 - 안검내반증
        '/content/val/dog/eye/normal/5/yu',
        '/content/val/dog/eye/normal/6/mu', # 6 - 안검염
        '/content/val/dog/eye/normal/6/yu',
        '/content/val/dog/eye/normal/7/mu', # 7 - 안검종양
        '/content/val/dog/eye/normal/7/yu',
        '/content/val/dog/eye/normal/8/mu', # 8 - 유루증
        '/content/val/dog/eye/normal/8/yu',
        '/content/val/dog/eye/normal/9/mu', # 9 - 핵경화
        '/content/val/dog/eye/normal/9/yu',
]

In [ ]:
!unzip -qq "/content/drive/MyDrive/PetData_Eye/1.Training/labelingdata/TL1/TL1.zip" # 구글 드라이브에 zip 파일을 저장한 path는 각자 확인해야한다.
!unzip -qq "/content/drive/MyDrive/PetData_Eye/1.Training/labelingdata/TL2/TL2.zip" # 구글 드라이브에 zip 파일을 저장한 path는 각자 확인해야한다.
os.makedirs('/content/val', exist_ok=True)
!unzip -qq /content/drive/MyDrive/PetData_Eye/2.Validation/labelingdata/VL/VL.zip -d /content/val # 구글 드라이브에 zip 파일을 저장한 path는 각자 확인해야한다.
print("job done_" + "unzip")

job done_unzip


In [ ]:
# unzip 한 파일의 경로에 한글이 섞여있으니 영어와 숫자로 변경
# 개관련 데이터만 경로를 영어로 바꾼다
os.rename('/content/개', '/content/dog')
os.rename('/content/dog/안구', '/content/dog/eye')
os.rename('/content/dog/eye/일반', '/content/dog/eye/normal')
normal = '/content/dog/eye/normal'
for folder in os.listdir(normal):
  if folder == "결막염":
    os.rename(normal+'/'+folder, normal+'/'+"0")
  if folder == "궤양성각막질환":
    os.rename(normal+'/'+folder, normal+'/'+"1")
  if folder == "백내장":
    os.rename(normal+'/'+folder, normal+'/'+"2")
  if folder == "비궤양성각막질환":
    os.rename(normal+'/'+folder, normal+'/'+"3")
  if folder == "색소침착성각막염":
    os.rename(normal+'/'+folder, normal+'/'+"4")
  if folder == "안검내반증":
    os.rename(normal+'/'+folder, normal+'/'+"5")
  if folder == "안검염":
    os.rename(normal+'/'+folder, normal+'/'+"6")
  if folder == "안검종양":
    os.rename(normal+'/'+folder, normal+'/'+"7")
  if folder == "유루증":
    os.rename(normal+'/'+folder, normal+'/'+"8")
  if folder == "핵경화":
    os.rename(normal+'/'+folder, normal+'/'+"9")

# 무 -> mu
for folder in os.listdir(normal):
  for state in os.listdir(normal+'/'+folder):
    src = os.path.join(normal+'/'+folder, state) # 기존 파일 경로
    if state == '무':
      dst = os.path.join(normal+'/'+folder+'/', "mu") # 바뀐 이름으로 저장할 경로
    if state == '유':
      dst = os.path.join(normal+'/'+folder+'/', "yu")
    if state == '상':
      dst = os.path.join(normal+'/'+folder+'/', "san")
    if state == '하':
      dst = os.path.join(normal+'/'+folder+'/', "ha")
    if state == '초기':
      dst = os.path.join(normal+'/'+folder+'/', "choGi")
    if state == '성숙':
      dst = os.path.join(normal+'/'+folder+'/', "seungSuk")
    if state == '비성숙':
      dst = os.path.join(normal+'/'+folder+'/', "beSeungSuk")
    os.rename(src, dst)

####################### val
os.rename('/content/val/개', '/content/val/dog')
os.rename('/content/val/dog/안구', '/content/val/dog/eye')
os.rename('/content/val/dog/eye/일반', '/content/val/dog/eye/normal')
normal = '/content/val/dog/eye/normal'
## 병명 -> 0,1,2,3...

for folder in os.listdir(normal):
  if folder == "결막염":
    os.rename(normal+'/'+folder, normal+'/'+"0")
  if folder == "궤양성각막질환":
    os.rename(normal+'/'+folder, normal+'/'+"1")
  if folder == "백내장":
    os.rename(normal+'/'+folder, normal+'/'+"2")
  if folder == "비궤양성각막질환":
    os.rename(normal+'/'+folder, normal+'/'+"3")
  if folder == "색소침착성각막염":
    os.rename(normal+'/'+folder, normal+'/'+"4")
  if folder == "안검내반증":
    os.rename(normal+'/'+folder, normal+'/'+"5")
  if folder == "안검염":
    os.rename(normal+'/'+folder, normal+'/'+"6")
  if folder == "안검종양":
    os.rename(normal+'/'+folder, normal+'/'+"7")
  if folder == "유루증":
    os.rename(normal+'/'+folder, normal+'/'+"8")
  if folder == "핵경화":
    os.rename(normal+'/'+folder, normal+'/'+"9")

# 무 -> mu
for folder in os.listdir(normal):
  for state in os.listdir(normal+'/'+folder):
    src = os.path.join(normal+'/'+folder, state) # 기존 파일 경로
    if state == '무':
      dst = os.path.join(normal+'/'+folder+'/', "mu") # 바뀐 이름으로 저장할 경로
    if state == '유':
      dst = os.path.join(normal+'/'+folder+'/', "yu")
    if state == '상':
      dst = os.path.join(normal+'/'+folder+'/', "san")
    if state == '하':
      dst = os.path.join(normal+'/'+folder+'/', "ha")
    if state == '초기':
      dst = os.path.join(normal+'/'+folder+'/', "choGi")
    if state == '성숙':
      dst = os.path.join(normal+'/'+folder+'/', "seungSuk")
    if state == '비성숙':
      dst = os.path.join(normal+'/'+folder+'/', "beSeungSuk")
    os.rename(src, dst)


In [ ]:
# colab vm 로 돌릴때용 폴더 세팅
# 입력 폴더 및 출력 폴더 설정
input_folder = '/content/dog/'
driver_root = '/content/Relabel_Dog/'
for i in range(0,10):
  test_folder_images = driver_root + str(i) + '/test/images'
  test_folder_labels = driver_root + str(i) + '/test/labels'
  os.makedirs(test_folder_labels, exist_ok=True)
  os.makedirs(test_folder_images, exist_ok=True)
  train_folder_images = driver_root + str(i) + '/train/images'
  train_folder_labels = driver_root + str(i) + '/train/labels'
  os.makedirs(train_folder_labels, exist_ok=True)
  os.makedirs(train_folder_images, exist_ok=True)
  valid_folder_images = driver_root + str(i) + '/val/images'
  valid_folder_labels = driver_root + str(i) + '/val/labels'
  os.makedirs(valid_folder_labels, exist_ok=True)
  os.makedirs(valid_folder_images, exist_ok=True)


In [ ]:
# # train 폴더에 모두 옮기기
# for input_dog in dog_list_train_vm_2:
#   i = input_dog.split('/')[-2]
#   output_folder_label = driver_root + str(i) +'/train/labels'
#   output_folder_images = driver_root + str(i) + '/train/images'
#   print("job start_" + str(i))
#   json_to_txt_And_copy_images_Dog(input_dog, output_folder_label, output_folder_images)
#   #### json_to_yoloLabelTxt_dog_BigSmall(input_dog, output_folder_label)
#   print("job done_" + str(i))

# 위의 코드를 사용하면, 모든 질병의 데이터를 한번에 처리하면 오래 걸리므로
# 각 질병의 무/유 폴더의 파일을 옮길땐 아래의 코드를 사용
####################################
input_dog = dog_list_train_vm_2[22]
i = input_dog.split('/')[-2]
output_folder_label = driver_root + str(i) +'/train/labels'
output_folder_images = driver_root + str(i) + '/train/images'
json_to_txt_And_copy_images_Dog(input_dog, output_folder_label, output_folder_images)
print("job_done" + input_dog)
####################################
input_dog = dog_list_train_vm_2[23]
i = input_dog.split('/')[-2]
output_folder_label = driver_root + str(i) +'/train/labels'
output_folder_images = driver_root + str(i) + '/train/images'
json_to_txt_And_copy_images_Dog(input_dog, output_folder_label, output_folder_images)
print("job_done" + input_dog)


job_done/content/dog/eye/normal/9/mu
job_done/content/dog/eye/normal/9/yu


In [ ]:
# val 폴더에 모두 옮기기
# for input_dog in dog_list_valid_vm_2:
#   i = input_dog.split('/')[-2]
#   output_folder_label = driver_root + i + '/val/labels'
#   output_folder_images = driver_root + i + '/val/images'
#   print("job start_" + str(i))
#   json_to_txt_And_copy_images_Dog(input_dog, output_folder_label, output_folder_images)
#   #### json_to_yoloLabelTxt_dog_BigSmall(input_dog, output_folder_label)
#   print("job done_" + str(i))
###############
input_dog = dog_list_valid_vm_2[22]
i = input_dog.split('/')[-2]
output_folder_label = driver_root + str(i) +'/val/labels'
output_folder_images = driver_root + str(i) + '/val/images'
json_to_txt_And_copy_images_Dog(input_dog, output_folder_label, output_folder_images)
print("job_done" + input_dog)
###########
input_dog = dog_list_valid_vm_2[23]
output_folder_label = driver_root + str(i) +'/val/labels'
output_folder_images = driver_root + str(i) + '/val/images'
json_to_txt_And_copy_images_Dog(input_dog, output_folder_label, output_folder_images)
print("job_done" + input_dog)

job_done/content/val/dog/eye/normal/9/mu
job_done/content/val/dog/eye/normal/9/yu


In [ ]:
# train:test:val = 8 : 0 : 1 비율이다.
# train:test =  7:1 로 split하고 옮기기
# for i in range(0,10):
#   before_split_folder = "/content/Relabel_Dog/" + str(i) + "train/images"
#   after_split = "/content/Relabel_Dog/" + str(i) + "/test/images"
#   split_data(before_split_folder, after_split, ratio=0.875)
#   print("job done_images" + str(i))
#   before_split_folder = "/content/Relabel_Dog/" + str(i) + "/train/labels"
#   after_split = "/content/Relabel_Dog/" + str(i) + "/test/labels"
#   split_data(before_split_folder, after_split, ratio=0.875)
#   print("job done_labels" + str(i))

############################
before_split_folder = "/content/Relabel_Dog/" + str(i) + "/train/images"
after_split = "/content/Relabel_Dog/"+ str(i) + "/test/images"
split_data(before_split_folder, after_split, ratio=0.875)
print("job done_images")
before_split_folder = "/content/Relabel_Dog/" + str(i) + "/train/labels"
after_split = "/content/Relabel_Dog/" + str(i) + "/test/labels"
split_data(before_split_folder, after_split, ratio=0.875)
print("job done_labels")

job done_images
job done_labels


In [ ]:
###############################################

In [ ]:
# 가끔씩 utf-8에서 다른 걸로 바뀔수가 있는데 그러면 일부 패키지등이 설치가 안된다
# 아래의 코드를 실행하면 다시 utf-8로 인코딩 방식을 지정
import locale
print(locale.getpreferredencoding())
def getpreferredencoding(do_setlocale = True):
  return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
print(locale.getpreferredencoding())

UTF-8
UTF-8


In [ ]:
# YOLOv8을 사용하기 위한 패키지를 설치
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.2/661.2 kB 8.1 MB/s eta 0:00:00


In [ ]:
import ultralytics
ultralytics.checks()
# 코랩환경인 파이썬 3.10에서 사용하기 위한 torch, cuda 등이 설치된걸 확인 할 수 있다.

Ultralytics YOLOv8.0.228 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 54.9/166.8 GB disk)


In [ ]:
from ultralytics import YOLO
from IPython.display import Image # 전처리용 PIL의 Image랑 이름이 겹치니까 모델 돌리기 이전에만 하자
from IPython import display
import cv2
display.clear_output()
!yolo mode=checks

Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 395, in entrypoint
    raise ValueError(f"Invalid 'mode={mode}'. Valid modes are {MODES}.\n{CLI_HELP_MSG}")
ValueError: Invalid 'mode=<module 'ultralytics.utils.checks' from '/usr/local/lib/python3.10/dist-packages/ultralytics/utils/checks.py'>'. Valid modes are ('train', 'val', 'predict', 'export', 'track', 'benchmark').

    Arguments received: ['yolo', 'mode=checks']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of ('detect', 'segment', 'classify', 'pose')
                MODE (required) is one of ('train', 'val', 'predict', 'export', 'track', 'benchmark')
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at https://d

In [ ]:
model = YOLO("yolov8n.pt") # custom dataset으로 학습할 때는 m,l 같은 모델을 불러와도 처음부터 n으로 시작한다.
results_9 = model.train(data='/content/Relabel_Dog/9/data_vm_0_10.yaml', epochs=50, imgsz=416)
# 만약 리사이즈 시에 416*416이 아닌 다른 크기로 했다면 imgsz를 해당 크기의 값으로 줘야한다.
# yaml 안의 train, test, val의 path는 지금까지의 경로명를 지정 및 수정하는 코드를 그대로 사용했다면 바로 쓸 수 있을 것이다.
# /content/Relabel_Dog/9/data_vm_0_10.yaml 에 꼭 넣을 yaml 파일을 넣을 필요는 없으며 /content/data_vm_0_10.yaml에 있어도 정상적으로 사용 가능하다.
# results_9 = model.train(data='/content/data_vm_0_10.yaml', epochs=50, imgsz=416)
# 만약 코랩의 vm환경에 yaml파일을 업로드 하고 바로 학습 시키고 싶다면 위의 줄에 있는 코드로 학습시키자

100%|██████████| 6.23M/6.23M [00:00<00:00, 124MB/s]


Ultralytics YOLOv8.0.228 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/Relabel_Dog/9/data_vm_0_10.yaml, epochs=50, time=None, patience=50, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_

100%|██████████| 755k/755k [00:00<00:00, 25.4MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/Relabel_Dog/9/train/labels... 14722 images, 2077 backgrounds, 0 corrupt: 100%|██████████| 16799/16799 [00:11<00:00, 1525.81it/s]


...

      50/50      1.04G     0.5218     0.8062      1.155         14        416: 100%|██████████| 1050/1050 [04:07<00:00,  4.24it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 75/75 [00:21<00:00,  3.51it/s]

                   all       2399       2399      0.648      0.389      0.414      0.393



50 epochs completed in 3.933 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.0.228 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 75/75 [00:25<00:00,  2.90it/s]


                   all       2399       2399      0.691      0.391      0.414      0.393
            No disease       2399       1201      0.383     0.0167      0.035    0.00954
     nuclear hardening       2399       1198      0.998      0.765      0.792      0.776
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/train
